In [2]:
import cv2
import hashlib
import os
import random

In [3]:
frame_loc = "Carta-preproc/frames-dev"
files = sorted(os.listdir(frame_loc))
outputs = [6, 4, 6, 4, 6]
num_files = len(files)

In [4]:
frame_num = 0
last = False
commit_list = []

while not last:
    image_file = f"{frame_loc}/{files[frame_num]}"
    img = cv2.imread(image_file)
    img_encode = cv2.imencode('.jpg', img)[1]
    img_bytes = img_encode.tobytes()
    img_uuid = hashlib.sha256(img_bytes).hexdigest()
    value = outputs[frame_num]
    commit_uuid = f"{img_uuid}_{value}"
#     commit_uuid = hashlib.sha256(commit.encode("utf-8")).hexdigest()   
    commit_list.append(commit_uuid)
    
#     print(f"img_uuid: {img_uuid}")
#     print(f"commit_uuid: {commit_uuid}\n")
    frame_num += 1
    if frame_num == num_files:
        last = True
print(commit_list)

['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6', 'd91ec2b1dfd69c916f2f6fdd314afff239cba45fc93fffadb4c6f7b4f347233e_4', 'eceb5b08f4b667568db75ccf6f1a260905c6babc7dcc6530f61f0c73f29c72f7_6', '6a92de3dfc6a50591ca747c69a637d6f7f52ebf456731dde4c890cb252f5fe8d_4', '9a349c9744350c288628856a6a8987d3c8757946b6daf738eb6024ac805309c4_6']


In [5]:
a = "6a92de3dfc6a50591ca747c69a637d6f7f52ebf456731dde4c890cb252f5fe8d"
hash_a = hashlib.sha256(str(a).encode("utf-8")).hexdigest()
print(hash_a)

3f20109eabdd5f1ca0e0a5c36aec7bb43b604322139ece03c9f1681d5cc66d28


In [6]:
output_checklist = []
hashed_output_checklist = hashlib.sha256(str(output_checklist).encode("utf-8")).hexdigest()
print(hashed_output_checklist)

4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945


In [7]:
random.seed(0)
for i in range(5):
    print(random.uniform(0,1))

0.8444218515250481
0.7579544029403025
0.420571580830845
0.25891675029296335
0.5112747213686085


In [23]:
import pulsar
import time
customer_uuid = "customer_ff2fc7dede4a441f9ac1823168bf35aa"
market_uuid = "market_13"
allocation_uuid = "allocation_0"

class InputDataSchema(pulsar.schema.Record):
    allocation_uuid = pulsar.schema.String()
    value = pulsar.schema.Bytes()
    timestamp = pulsar.schema.Float()
    msgnum = pulsar.schema.Integer()
    input_uuid = pulsar.schema.String()
    
class OutputDataSchema(pulsar.schema.Record):
    allocation_uuid = pulsar.schema.String()
    value = pulsar.schema.Integer()
    # customer = pulsar.schema.String()
    # service_name = pulsar.schema.String()
    # jobid = pulsar.schema.String()
    # start = pulsar.schema.Float()
    # end = pulsar.schema.Float()
    # supplier = pulsar.schema.String()
    # allocationid = pulsar.schema.String()
    customertimestamp = pulsar.schema.Float()
    suppliertimestamp = pulsar.schema.Float()
    msgnum = pulsar.schema.Integer()
    input_uuid = pulsar.schema.String()
    input_msgID = pulsar.schema.Bytes()
    timestamp = pulsar.schema.Float()

class CleanupDataSchema(pulsar.schema.Record):
    allocation_uuid = pulsar.schema.String()
    cleanup = pulsar.schema.Boolean
    timestamp = pulsar.schema.Float()

# consumer output_topic persistent://customer_31bb908ac1f64300845f25ae8ab11e4f/market_9/mediation_output_allocation_0 
# producer output_topic persistent://customer_31bb908ac1f64300845f25ae8ab11e4f/market_9/mediation_output_allocation_0

    
class mediator:
    def __init__(self):
        self.input_MessageId_list = [b'\x08\x8a\x05\x10\x00', b'\x08\x8a\x05\x10\x01', b'\x08\x8a\x05\x10\x02', b'\x08\x8a\x05\x10\x03', b'\x08\x8a\x05\x10\x04']
        self.pulsar_client = pulsar.Client(f"pulsar://172.21.20.70:6650")
        self.output_checklist = []
    def readTopic(self):
        for serial_msg_id in self.input_MessageId_list:
            msg_id = pulsar.MessageId.deserialize(serial_msg_id)
#             print(msg_id)
            reader = self.pulsar_client.create_reader(
                topic=f"persistent://{customer_uuid}/{market_uuid}/output_{allocation_uuid}",
                start_message_id=msg_id,
                schema=pulsar.schema.AvroSchema(InputDataSchema)
                )
            input_msg = reader.read_next()
            print(msg_id)
            print(input_msg.message_id())
#             time.sleep(.2)
            # app_input = input_msg.value().value
#             print(f"\033[91m send image: {input_msg.value().input_uuid} \033[00m")
    def getOutput(self, user_uuid):
        self.output_consumer = self.pulsar_client.subscribe(
#             topic=f"persistent://customer_31bb908ac1f64300845f25ae8ab11e4f/market_9/mediation_output_allocation_0",
            topic=f"persistent://customer_ff2fc7dede4a441f9ac1823168bf35aa/market_13/mediation_output_allocation_0",
            schema=pulsar.schema.AvroSchema(OutputDataSchema),
            subscription_name=f"{user_uuid}-output_{allocation_uuid}",
            initial_position=pulsar.InitialPosition.Earliest,
            consumer_type=pulsar.ConsumerType.Exclusive,
            message_listener=self.store_output)
        
    def getCleanup(self, user_uuid):
        self.output_consumer = self.pulsar_client.subscribe(
#             topic=f"persistent://customer_31bb908ac1f64300845f25ae8ab11e4f/market_9/mediation_output_allocation_0",
            topic=f"persistent://customer_b61ab3e3aa574f76ad229aa3e16978cd/market_10/mediation_cleanup_allocation_0",
            schema=pulsar.schema.AvroSchema(CleanupDataSchema),
            subscription_name=f"{user_uuid}-cleanup_{allocation_uuid}",
            initial_position=pulsar.InitialPosition.Earliest,
            consumer_type=pulsar.ConsumerType.Exclusive,
            message_listener=self.cleanup)
    
    def cleanup(self, consumer, msg):
        print("cleanup called")
        
    def store_output(self, consumer, msg):
        consumer.acknowledge_cumulative(msg)

        output = msg.value().value
        input_uuid = msg.value().input_uuid
        output_uuid = f"{input_uuid}_{output}"

        self.output_checklist.append(output_uuid)

        received = len(self.output_checklist)
        expected = len(self.input_MessageId_list)
        all_received = received >= expected

        print(f"\033[91m store_output: {self.output_checklist} < {len(self.input_MessageId_list)}"
              f"{all_received} \033[00m")

        if all_received:
            print("recieved all")
            print(f"\033[91m expected: {expected} received: {received} \033[00m ")
#             recv_timestamp = datetime.datetime.now(tz=datetime.timezone.utc).timestamp()
#             cleanup = MV2.schema.CleanupDataSchema(
#                 allocation_uuid=msg.value().allocation_uuid,
#                 cleanup=True,
#                 timestamp=recv_timestamp
#             )
#             self.cleanup_producer.send(cleanup)
        else:
            print(f"\033[91m expected: {expected} received: {received} \033[00m ")

m = mediator()
m.getOutput("m1")
# m.getCleanup("m2")

 store_output: ['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6'] < 5False 
 expected: 5 received: 1  
 store_output: ['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6', 'd91ec2b1dfd69c916f2f6fdd314afff239cba45fc93fffadb4c6f7b4f347233e_4'] < 5False 
 expected: 5 received: 2  
 store_output: ['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6', 'd91ec2b1dfd69c916f2f6fdd314afff239cba45fc93fffadb4c6f7b4f347233e_4', 'eceb5b08f4b667568db75ccf6f1a260905c6babc7dcc6530f61f0c73f29c72f7_6'] < 5False 
 expected: 5 received: 3  
 store_output: ['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6', 'd91ec2b1dfd69c916f2f6fdd314afff239cba45fc93fffadb4c6f7b4f347233e_4', 'eceb5b08f4b667568db75ccf6f1a260905c6babc7dcc6530f61f0c73f29c72f7_6', '6a92de3dfc6a50591ca747c69a637d6f7f52ebf456731dde4c890cb252f5fe8d_4'] < 5False 
 expected: 5 received: 4  
 store_output: ['7ed7e492467ccad8dc2dd8876e9e0725867a3064b2903961bba2884fdc68a66b_6', 'd